# Tensorflow

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

  print('TPU devices:')
  pprint.pprint(devices)

ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=3c39a66f29415b7ca3c129adb1d011df3fc3957cda069ccc9e0ab64fd375a147
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 11.5 GB  | Proc size: 1.3 GB
GPU RAM Free: 14768MB | Used: 333MB | Util   2% | Total 15360MB


# Train n-gram

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
batch_size = 2048
from collections import defaultdict, Counter

def train_fourgram(tokens, fourgram_counts):
    batches = [tokens[i:i+batch_size] for i in range(0, len(tokens), batch_size)]
    for batch in batches:
        for i in range(len(batch)-3):
            context = tuple(batch[i:i+3])
            word = batch[i+3]
            fourgram_counts[context][word] += 1
    return fourgram_counts

def create_fourgram_model(fourgram_counts):
    model = defaultdict(Counter)
    for context, counts in fourgram_counts.items():
        total_count = sum(counts.values())
        for word, count in counts.items():
            prob = count / total_count
            model[context][word] = prob
    return model

with open('/content/drive/MyDrive/com ling/contest 2/contest 2/train.src.tok', 'r') as f:
    counts_dict = defaultdict(Counter)
    chunk_size = 50000
    tokens = []
    for i, line in enumerate(f):
        line_tokens = line.strip().split() # tokenize the line
        tokens.extend(line_tokens) # add tokens to list
        if len(tokens) > chunk_size:
            # process the chunk of tokens
            counts_dict = train_fourgram(tokens, counts_dict)
            # reset the tokens list for the next chunk
            tokens = []
    # process the remaining tokens
    if len(tokens) > 0:
        counts_dict = train_fourgram(tokens, counts_dict)

# create the 4-gram model from the counts dictionary
model = create_fourgram_model(counts_dict)


In [ ]:
len(counts_dict)

In [ ]:
def predict_word(fourgram_context1, fourgram_context2, fourgram_context3, counter, first_letter):
    if (fourgram_context1, fourgram_context2, fourgram_context3) in counter:
        # get all the words in the given context that start with the given first letter
        words = [word for word in counter[fourgram_context1, fourgram_context2, fourgram_context3] if word[0] == first_letter]
        if len(words) > 0:
            # sort the words by their frequency and return the most common word
            return max(words, key=counter[fourgram_context1, fourgram_context2, fourgram_context3].get)
        else:
            # if no words match the given first letter, return an empty string
            return ''
    else:
        # return an empty string to represent an unknown context
        return ''

In [ ]:
next_word = predict_word('the', 'next','month' model, 'i')
print(next_word)

# Evaluate

In [ ]:
import pandas as pd
dev = pd.read_csv('/content/drive/MyDrive/com ling/contest 2/contest 2/dev_set.csv')

In [ ]:
dev['tokens'] = dev.context.apply(lambda x: x.split())
dev

In [ ]:
dev_set = dev[['tokens', 'first letter']]
dev_set

In [ ]:
prediction = []
for i, row in dev_set.iterrows():
  tokens = row['tokens']
  possible_first_letter = row['first letter']
  next_word = predict_word(tokens[-3],tokens[-2], tokens[-1], model, possible_first_letter)
  prediction.append(next_word)
print(prediction[:100])

In [ ]:
dev_set['prediction'] = prediction

In [ ]:
def accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1

    acc = correct / len(actual)
    return acc

In [ ]:
actual = dev['answer']
predicted = dev_set['prediction']

In [ ]:
accuracy(actual, predicted)